In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qaeueag6
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qaeueag6
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=3674e83786c5e2a9f2815756e962910603144707c7fc55e4abd4819d62b4891d
  Stored in directory: /tmp/pip-ephem-wheel-cache-yhlqe06l/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%reload_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cu 
#include <stdio.h>
#include <stdlib.h>

//have to take this in row major format


#define N 2

void generateRandomArray(int* arrayA,int *arrayB) {
	srand((unsigned) time(NULL));
	for(int i = 0; i<N*N; ++i){
		arrayA[i] = ((int)rand()/(int)(RAND_MAX)) * 100;
		arrayB[i] = ((int)rand()/(int)(RAND_MAX)) * 100;
		
	}
}

__global__ void matrixAddRowOnly(int *A,int *B,int *C){
    int rowId = threadIdx.y+blockIdx.y*blockDim.y;

    if(rowId <2){
        for(int i=0;i<2;i++){
            C[i+rowId*2] = A[i+rowId*2]+B[i+rowId*2];
        }
    }

}

int main(){
    //in actuality a 2*2 matrix

    const size_t d_size = sizeof(int) * size_t(N*N);

    int *A = (int*) malloc(N*N * sizeof(int));
	  int *B = (int*) malloc(N*N * sizeof(int));
	  int *C = (int*) malloc(N*N * sizeof(int));
    //generateRandomArray(A,B);
    for(int i=0;i<N*N;i++){
        A[i]=2*i+1;
    }
    for(int i=0;i<N*N;i++){
        B[i]=3*i+2;
    }
    for(int i=0;i<N*N;i++){
        C[i]=0;
    }

    int *pA , *pB,*pC;
    cudaMalloc((void**)&pA,d_size);
    cudaMalloc((void**)&pB,d_size);
    cudaMalloc((void**)&pC,d_size);

    cudaMemcpy(pA,A,d_size,cudaMemcpyHostToDevice);
    cudaMemcpy(pB,B,d_size,cudaMemcpyHostToDevice);
    cudaMemcpy(pC,C,d_size,cudaMemcpyHostToDevice);
    int numBlocks=1; //cause using single block
    dim3 threadsPerBlock(N,N);
    dim3 blocksPerGrid(1,1);
    matrixAddRowOnly<<<blocksPerGrid,threadsPerBlock>>>(pA,pB,pC);
    cudaMemcpy(C,pC,d_size,cudaMemcpyDeviceToHost);
    for(int i=0;i<N*N;i++){
        printf("A: =%d, ",A[i]);
    }
    printf("\n");
    for(int i=0;i<N*N;i++){
        printf("B: =%d, ",B[i]);
    }
    printf("\n");
    for(int i=0;i<N*N;i++){
        printf("C: =%d, ",C[i]);
    }
   

    cudaFree(pA);
    cudaFree(pB);
    cudaFree(pC);
    free(A);
    free(B);
    free(C);
    return 0;

}

A: =1, A: =3, A: =5, A: =7, 
B: =2, B: =5, B: =8, B: =11, 
C: =3, C: =8, C: =13, C: =18, 
